In [1]:
import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
appName = "Python Example - PySpark Read CSV"
master = 'local'

# Create Spark session
spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

df = spark.read.option("header",True).csv("/project/input/NJ_Parking.csv")

df.show(2)
df.printSchema()

+---+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+--------+---------+---------------+------------------+------------+----+----+----+
|_c0|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Co

In [5]:
df.select('Summons Number').describe().show()

+-------+-------------------+
|summary|     Summons Number|
+-------+-------------------+
|  count|             878677|
|   mean|6.744945115569269E9|
| stddev|2.022111483621511E9|
|    min|         1002891188|
|    max|         8006050284|
+-------+-------------------+



In [6]:
df.select('Summons Number').describe().toPandas()

,summary,Summons Number
0,count,878677
1,mean,6.744945115569269E9
2,stddev,2.022111483621511E9
3,min,1002891188
4,max,8006050284


In [7]:
df.select('Summons Number','Plate ID',
'Registration State',
'Plate Type',
'Issue Date',
'Violation Code',
'Vehicle Body Type').describe().toPandas()

,summary,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type
0,count,878677,878676,878677,878677,878677,878677,872518
1,mean,6.744945115569269E9,Infinity,None,None,None,32.315087341537335,3.739130434782609
2,stddev,2.022111483621511E9,NaN,None,None,None,18.96446679869914,0.6887004431501819
3,min,1002891188,0%67^N,NJ,PAS,01/01/2013,0,2 D
4,max,8006050284,ZZZ99R,NJ,PAS,12/31/2013,99,Z


In [8]:
df.select('Summons Number').describe().show()

+-------+-------------------+
|summary|     Summons Number|
+-------+-------------------+
|  count|             878677|
|   mean|6.744945115569269E9|
| stddev|2.022111483621511E9|
|    min|         1002891188|
|    max|         8006050284|
+-------+-------------------+



In [9]:
from pyspark.mllib.stat import Statistics

# select variables to check correlation
df_features = df.select('Summons Number','Plate ID',
'Plate Type',
'Violation Code',
'Vehicle Body Type') 

# create RDD table for correlation calculation
rdd_table = df_features.rdd.map(lambda row: row[0:])

# get the correlation matrix
corr_mat=Statistics.corr(rdd_table, method="pearson")

Py4JJavaError: An error occurred while calling o91.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 14, 192.168.1.5, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 587, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 74, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 172, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 458, in loads
    return pickle.loads(obj, encoding=encoding)
  File "<frozen zipimport>", line 259, in load_module
  File "/opt/spark/python/lib/pyspark.zip/pyspark/mllib/__in

In [10]:
display(df.select('Summons Number'))

DataFrame[Summons Number: string]

In [11]:
df1=df.toPandas()

In [12]:
df1.head()

,_c0,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,5,1283294217,T60DAR,NJ,PAS,08/11/2013,14,P-U,DODGE,P,...,None,None,None,None,None,None,None,None,None,None
1,7,1283983620,XZ764G,NJ,PAS,08/07/2013,24,DELV,FORD,X,...,None,None,None,None,None,None,None,None,None,None
2,9,1283983667,MCL78B,NJ,PAS,07/18/2013,24,SDN,SUBAR,H,...,None,None,None,None,None,None,None,None,None,None
3,22,1286654920,ZTR66R,NJ,PAS,07/18/2013,50,SDN,N/S,T,...,None,None,None,None,None,None,None,None,None,None
4,47,1291896399,G24BLB,NJ,PAS,07/19/2013,40,SDN,INFIN,P,...,None,None,None,None,None,None,None,None,None,None
